In [1]:
import numpy
import matplotlib.pyplot as plt
import pandas as pd
import autograd.numpy as np
from autograd import elementwise_grad

In [2]:
def cost_function(theta,delta=1):
    error = ((y - np.dot(X,theta))**2)/len(X) + delta**2*(numpy.linalg.norm(theta,1))
    return error

def normalEquationRidgeRegression(x, y, delta=1):
    X = np.c_[np.ones(len(x)),x]
    temp1 = (np.transpose(X).dot(X) + delta**2*np.eye(X.shape[1]))
    return np.linalg.inv(temp1).dot(np.transpose(X)).dot(y)

def coordinateDescentRegression(x,y,iteration=1000):
    X = np.c_[np.ones(len(x)),x]
    thetas = np.random.rand(len(X[0]))
    for it in range(iteration):
        for j in range(len(thetas)):
            temp1 = 0
            zj = 0
            temp2 = 0
            for i in range(len(y)):
                zj+=(X[i][j])**2
                temp1+=y[i]*X[i][j]
                for k in range(len(thetas)):
                    if k!=j:
                        temp2+=thetas[k]*X[i][k]*X[i][j]
            rhoj = temp1 - temp2
            thetas[j] = rhoj/zj
    return thetas

def coordinateDescentLasso(x,y,iteration=1000,delta=1):
    X = np.c_[np.ones(len(x)),x]
    thetas = np.random.rand(len(X[0]))
    for it in range(iteration):
        for j in range(len(thetas)):
            temp1 = 0
            zj = 0
            temp2 = 0
            for i in range(len(y)):
                zj+=(X[i][j])**2
                temp1+=y[i]*X[i][j]
                for k in range(len(thetas)):
                    if k!=j:
                        temp2+=thetas[k]*X[i][k]*X[i][j]
            rhoj = temp1 - temp2
            if rhoj<-(delta**2)/2:
                thetas[j] = (rhoj + ((delta**2)/2))/zj
            if rhoj>= (-delta**2)/2 and rhoj<=(delta**2)/2:
                thetas[j] = 0
            if rhoj> (delta**2)/2:
                thetas[j] = (rhoj - (delta**2)/2)/zj
    return thetas

def sgdRegression(x,y,alpha=0.1,epoch=1000):
    X = np.c_[np.ones(len(x)),x]
    thetas = np.random.rand(len(X[0]))
    for it in range(epoch):
        for i in range(len(y)):
            old = np.copy(thetas)
            for j in range(len(thetas)):
                old[j] = old[j] + 2*alpha*(y[i] - X[i].dot(thetas))*X[i][j]
            thetas = old
    return thetas

def gradientDescentAutogradLasso(x,y,alpha = 0.1, delta = 1,iteration=10000):
    X = np.c_[np.ones(len(x)),x]
    theta_old = np.random.rand(len(X[0]))
    agrad = elementwise_grad(cost_function)
    for i in range(iteration):
        val = agrad(theta_old,1)
        predicted = X.dot(theta_old)
        error = y - predicted
        temp = theta_old - alpha*val
        theta_old = temp
    return theta_old

                

In [3]:
x = np.array([1,3,6])
y = np.array([106,110,116])
X = np.c_[np.ones(len(x)),x]
print("Normal Equation Thetas are ",normalEquationRidgeRegression(x,y,1))
print("Coordinate (w/o) regularisation thetas are ",coordinateDescentRegression(x,y))
print("Coordinate (w/) regularisation thetas are ",coordinateDescentLasso(x,y))
print("Stochastic Gradient Descent thetas are ",sgdRegression(x,y,alpha = 0.01))
print("Autograd Solutions w/ regularisation are",gradientDescentAutogradLasso(x,y,alpha=0.01))

Normal Equation Thetas are  [48.68181818 13.72727273]
Coordinate (w/o) regularisation thetas are  [104.   2.]
Coordinate (w/) regularisation thetas are  [103.52631579   2.09210526]
Stochastic Gradient Descent thetas are  [103.99999641   2.00000065]
Autograd Solutions w/ regularisation are [99.73684211  2.82894737]


For Autograd Lasso, we do not get the correct solution - maybe the automatic differentiator doesn't work the way in which it should.